In [4]:
!wget 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'

--2023-06-12 09:14:34--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-06-12 09:14:35 (20.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [5]:
with open ('input.txt','r',encoding='utf-8') as f:
    text = f.read()
    
print(len(text))
print(text[0:300])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# create a mapping from charecters to integers
stoi = {ch:i for i,ch in enumerate(chars)} # str to int stoi
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s : [stoi[ch] for ch in s]
decode = lambda s : ''.join([itos[i] for i in s])

In [8]:
encode('hii there')

[46, 47, 47, 1, 58, 46, 43, 56, 43]

In [9]:
decode([46, 47, 47, 1, 58, 46, 43, 56, 43])

'hii there'

In [13]:
import torch 
data = torch.tensor(encode(text) , dtype = torch.long)

print(data.shape,data.dtype)
print(data[0:300])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [23]:
# splitting the data into train and test
n = int(.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [21]:
# context length
block_size = 8

In [22]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(8):
  context = x[:t+1]
  target = y[t]
  print(f'the context : {context} and target : {target}')

the context : tensor([18]) and target : 47
the context : tensor([18, 47]) and target : 56
the context : tensor([18, 47, 56]) and target : 57
the context : tensor([18, 47, 56, 57]) and target : 58
the context : tensor([18, 47, 56, 57, 58]) and target : 1
the context : tensor([18, 47, 56, 57, 58,  1]) and target : 15
the context : tensor([18, 47, 56, 57, 58,  1, 15]) and target : 47
the context : tensor([18, 47, 56, 57, 58,  1, 15, 47]) and target : 58


In [37]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split):
  data = train_data if split=='train' else val_data
  ix = torch.randint(len(data)-block_size , (batch_size,) )
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y


xb , yb = get_batch('train')

print('inputs : ')
print(xb)
print(xb.shape)

print('output : ')
print(yb)
print(yb.shape)


print('---')


for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f'when context is {context} the target is {target}')

inputs : 
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
output : 
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
torch.Size([4, 8])
---
when context is tensor([24]) the target is 43
when context is tensor([24, 43]) the target is 58
when context is tensor([24, 43, 58]) the target is 5
when context is tensor([24, 43, 58,  5]) the target is 57
when context is tensor([24, 43, 58,  5, 57]) the target is 1
when context is tensor([24, 43, 58,  5, 57,  1]) the target is 46
when context is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
when context is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
when context is tensor([44]) the target is 53
when context is tensor([44, 53]) the target is 56
when context is tensor([44

In [45]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a look up table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self,idx,targets):
    # idx is the inputs
    # idx and targets are both [B,T] tensor of integers
    logits = self.token_embedding_table(idx) # (B,T,C)
    B,T,C = logits.shape
    logits = logits.view(B*T , C)
    targets = targets.view(B*T) # or targets.view(-1)

    loss = F.cross_entropy(logits,targets)
    return logits, loss

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)

torch.Size([32, 65])


In [46]:
loss

tensor(4.8786, grad_fn=<NllLossBackward0>)

# stopped at 27 minutes of nanoGPT